In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import sinabs
from torchvision import transforms, datasets
import sinabs.layers as sl
import numpy as np
import quartz
from quartz.utils import get_accuracy, encode_inputs, decode_outputs, plot_output_histograms, plot_output_comparison, normalize_weights, plot_output_comparison_ann_snn
from mnist_model import ConvNet
from copy import deepcopy
from typing import List
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 200
num_workers = 4

transform = transforms.Compose([transforms.ToTensor()])
valid_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

In [ ]:
state_dict = torch.load("mnist-convnet.pth", map_location=torch.device(device))
ann = ConvNet()

ann[0].weight.data = state_dict['conv1.weight']
ann[0].bias.data = state_dict['conv1.bias']
ann[3].weight.data = state_dict['conv2.weight']
ann[3].bias.data = state_dict['conv2.bias']
ann[6].weight.data = state_dict['conv3.weight']
ann[6].bias.data = state_dict['conv3.bias']
ann[10].weight.data = state_dict['fc1.weight']
ann[10].bias.data = state_dict['fc1.bias']

ann.eval();
# get_accuracy(ann, valid_loader, device)

In [ ]:
norm_ann = deepcopy(ann)
param_layer_names = [name for name, child in norm_ann.named_children() if isinstance(child, (nn.Conv2d, nn.Linear))]
normalise_loader = DataLoader(dataset=valid_dataset, batch_size=10000, shuffle=True)
all_test_images = next(iter(normalise_loader))[0]
percentile = 99.99

In [ ]:
normalize_weights(norm_ann, all_test_images.to(device), param_layer_names=param_layer_names, percentile=percentile)

In [ ]:
output_layer_names = [name for name, child in norm_ann.named_children() if isinstance(child, nn.ReLU)]
output_layer_names += [param_layer_names[-1]]
# sinabs.utils.normalize_weights(norm_ann, sample_data.to(device), param_layers=param_layer_names, output_layers=output_layer_names, percentile=percentile)

In [ ]:
# get_accuracy(norm_ann, valid_loader, device)

In [ ]:
# plot_output_comparison(ann.to(device), norm_ann.to(device), sample_data.to(device), param_layer_names, every_n=100, savefig=f"norm_activation_correct_biases.png")

In [ ]:
# with torch.no_grad():
#     norm_ann[-1].weight *= 1.5
#     norm_ann[-1].bias *= 1.5
# get_accuracy(norm_ann, valid_loader, device)

In [ ]:
cpu = 'cpu'
t_max = 2**6

sample_data = next(iter(valid_loader))[0]

snn = quartz.from_torch.from_model(norm_ann, t_max=t_max, add_spiking_output=True).to(cpu).eval()
snn_output_layers = [name for name, child in snn.named_children() if isinstance(child, quartz.IF)]

In [ ]:
plot_output_comparison_ann_snn(norm_ann.to(cpu), snn.to(cpu), sample_data.to(cpu), 
    ann_output_layers=output_layer_names, 
    snn_output_layers=snn_output_layers, 
    t_max=t_max, 
    every_n=10, 
    every_c=1, 
    savefig=f"ann-snn-comparison-tmax{t_max}-percentile{percentile}.png"
)

In [ ]:
q_ann = deepcopy(norm_ann)
for exponent in range(1, 7):
    t_max = 2**exponent
    def quantize(module, input, output):
        return (output * t_max).round() / t_max

    for module in q_ann.children():
        if isinstance(module, nn.ReLU):
            module.register_forward_hook(quantize)
    q_ann[-1].register_forward_hook(quantize)

    accuracy = get_accuracy(q_ann, valid_loader, device)
    print(f"{t_max} time steps: {round(accuracy, 3)}%")

In [ ]:
accuracies = []
for exponent in range(1, 7):
    t_max = 2**exponent
    snn = quartz.from_torch.from_model(norm_ann, t_max=t_max, add_spiking_output=True).to(device).eval()
    accuracy = get_accuracy(snn, valid_loader, device, t_max=t_max, print_early_spikes=True, print_output_time=True)
    accuracies.append(accuracy)
    print(f"{t_max} time steps: {round(accuracy, 3)}%")

In [ ]:
accuracies

In [ ]:
t_max = 2**3+1
images, label = next(iter(valid_loader))
spikes = encode_inputs(images, t_max=t_max).to(device)
snn = quartz.from_torch.from_model(norm_ann, t_max=t_max, add_spiking_output=True).to(device).eval()
output_layers = [child for name, child in snn.named_children() if isinstance(child, sl.StatefulLayer)]

plot_output_histograms(snn, spikes, output_layers, t_max=t_max)

In [ ]:
t_max = 2**3+1
def quantize(module, input, output):
    return (output * t_max).int() / t_max

for module in ann.children():
    if isinstance(module, nn.ReLU):
        module.register_forward_hook(quantize)

param_layers = [child for name, child in ann.named_children() if isinstance(child, (nn.Conv2d, nn.Linear))]
output_layers = [child for name, child in ann.named_children() if isinstance(child, nn.ReLU)]
output_layers += [param_layers[-1]]
plot_output_histograms(ann, images.to(device), output_layers)

In [ ]:
snn

In [ ]:
t_max = 2**5
snn = quartz.from_torch.from_model(ann, t_max=t_max, add_spiking_output=True).to(device).eval()
synop_counter = sinabs.SNNSynOpCounter(snn)
accuracy = get_accuracy(snn, valid_loader, device, t_max=t_max)
print(f"{t_max} time steps: {round(accuracy, 3)}%")

In [ ]:
synop_counter.get_total_synops()

In [ ]:
synop_counter.get_total_synops()